# 🎤 Fine-Tuning StyleTTS2 pour le Darija

## Version Optimisée GitHub + HuggingFace (Setup rapide ~3 min)

**Sources:**
- 📦 **Code:** `github.com/VOTRE-USERNAME/arable-tts` ⬅️ Remplacez par votre username GitHub
- 🎵 **Audio:** `huggingface.co/datasets/VOTRE-USERNAME/darija-dataset` ⬅️ Remplacez par votre repo HF

### Prérequis:
- GPU: T4, V100, ou A100 (minimum 16GB VRAM)
- Dataset déjà uploadé sur Hugging Face

## 📝 Configuration - MODIFIEZ ICI

In [1]:
# 🔧 CONFIGURATION - Modifiez ces valeurs
GITHUB_REPO = "Racim679/tts"  # Votre repo GitHub

print(f"✅ GitHub: {GITHUB_REPO}")

✅ GitHub: Racim679/tts


## 1. Vérification GPU

In [2]:
!nvidia-smi
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Wed Dec 10 13:53:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Installation des dépendances

In [3]:
!pip install -q phonemizer==3.2.1 munch accelerate pydub nltk g2p_en num2words inflect unidecode pyyaml librosa scipy matplotlib soundfile
!pip install -q torch torchaudio transformers einops einops-exts tqdm omegaconf huggingface_hub
!pip install -q git+https://github.com/resemble-ai/monotonic_align.git
!apt-get install -qq espeak-ng
print("✅ Dépendances installées!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 33.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Selecting previously unselected package libpcaudio0:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to 

## 3. Téléchargement du code (GitHub) et audio (HuggingFace)

In [18]:
import os
from google.colab import drive

# Monter Google Drive si nécessaire
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 🚀 CLONAGE DU PROJET (Source Unique: Racim679/tts)
if not os.path.exists("/content/StyleTTS2"):
    print("🔄 Installation du code depuis Racim679/tts...")
    # On clone le repo principal dans un dossier temp
    !git clone https://github.com/Racim679/tts.git /content/temp_repo
    
    # On déplace le dossier StyleTTS2 (le code) à la racine de Colab
    # C'est ce dossier qui contient tout le code unifié
    !mv /content/temp_repo/StyleTTS2 /content/StyleTTS2
    
    # Nettoyage
    !rm -rf /content/temp_repo
    print("✅ Code installé avec succès !")
else:
    print("ℹ️ Code déjà présent")

# 📥 DONNÉES (Dataset)
drive_zip_path = "/content/drive/MyDrive/darija_dataset.zip"
local_dataset_path = "/content/darija_dataset"

if os.path.exists(drive_zip_path):
    print(f"📦 ZIP trouvé: {drive_zip_path}")
    if not os.path.exists(local_dataset_path):
        print("⏳ Extraction...")
        !unzip -q {drive_zip_path} -d {local_dataset_path}
        print("✅ Données extraites !")
    else:
        print("ℹ️ Données déjà extraites")
else:
    print("⚠️ ZIP 'darija_dataset.zip' introuvable sur Drive !")


ℹ️ StyleTTS2 déjà présent
ℹ️ Repo déjà présent
📦 ZIP trouvé sur Drive: /content/drive/MyDrive/darija_dataset.zip
⏳ Extraction en cours... (ça peut prendre 1-2 min)
✅ Données extraites depuis Drive!
🎵 1052 fichiers audio détectés
📄 metadata_train.csv présent
📄 metadata_eval.csv présent
📄 metadata.json présent


## 4. Téléchargement des modèles pré-entraînés

In [20]:
minimal_config = """log_dir: "."
save_freq: 1
device: "cuda"
epochs_1st: 200
epochs_2nd: 800
batch_size: 16
max_len: 200

model_params:
  n_token: 178

F0_path: "Utils/JDC/bst.t7"
ASR_config: "Utils/ASR/config.yml"
ASR_path: "Utils/ASR/epoch_00080.pth"
"""

with open('/content/StyleTTS2/Utils/ASR/config.yml', 'w') as f:
    f.write(minimal_config)

print("✅ Config ASR créé")

# Vérifier
with open('/content/StyleTTS2/Utils/ASR/config.yml', 'r') as f:
    print(f.read())


✅ Config ASR créé
log_dir: "."
save_freq: 1
device: "cuda"
epochs_1st: 200
epochs_2nd: 800
batch_size: 16
max_len: 200

model_params:
  n_token: 178

F0_path: "Utils/JDC/bst.t7"
ASR_config: "Utils/ASR/config.yml"
ASR_path: "Utils/ASR/epoch_00080.pth"



In [21]:
import os

# Vérifier que les fichiers de config existent et sont valides
asr_config_path = "/content/StyleTTS2/Utils/ASR/config.yml"
if os.path.exists(asr_config_path):
    with open(asr_config_path, 'r') as f:
        content = f.read()
        if len(content) > 0:
             print(f"✅ ASR config OK ({len(content)} bytes)")
        else:
             print("❌ ASR config est vide!")
else:
      print("❌ ASR config n'existe pas!")

✅ ASR config OK (233 bytes)


## 5. Patches PyTorch 2.6+ (CRITIQUE!)

In [22]:
import re, os

files_to_patch = [
    "/content/StyleTTS2/models.py",
    "/content/StyleTTS2/Utils/ASR/models.py",
    "/content/StyleTTS2/Utils/JDC/model.py",
    "/content/StyleTTS2/Utils/PLBERT/util.py",
    "/content/StyleTTS2/meldataset.py",
]

for filepath in files_to_patch:
    if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            content = f.read()
        if 'torch.load' in content and 'weights_only' not in content:
            new_content = re.sub(r'torch\.load\(([^)]+)\)', r'torch.load(\1, weights_only=False)', content)
            with open(filepath, 'w') as f:
                f.write(new_content)
            print(f"✅ Patched: {os.path.basename(filepath)}")
        else:
            print(f"ℹ️ OK: {os.path.basename(filepath)}")

print("\n✅ Patches PyTorch appliqués!")

ℹ️ OK: models.py
ℹ️ OK: models.py
ℹ️ OK: model.py
ℹ️ OK: util.py
ℹ️ OK: meldataset.py

✅ Patches PyTorch appliqués!


## 6. Préparation des données

In [23]:
import json, random, os, shutil

print("🔍 Recherche automatique des données...")

# 1. Identifier où sont les fichiers (supporte extraction racine ou dans un sous-dossier)
source_dir = None
possible_roots = ["/content/dataset_darija", "/content/darija_dataset", "/content"]

for path in possible_roots:
    if os.path.exists(os.path.join(path, "metadata.json")) and os.path.exists(os.path.join(path, "wavs")):
        source_dir = path
        break

if not source_dir:
    # Recherche recursive de secours si toujours pas trouvé
    print("⚠️ Recherche approfondie en cours...")
    for root, dirs, files in os.walk("/content"):
        if "metadata.json" in files and "wavs" in dirs:
            source_dir = root
            print(f"⚠️ Données trouvées dans un emplacement inhabituel : {source_dir}")
            break

if not source_dir:
    raise FileNotFoundError("❌ Impossible de trouver 'metadata.json' et le dossier 'wavs'. Vérifiez l'étape de décompression du ZIP.")

print(f"✅ Source détectée : {source_dir}")

# 2. Préparer les dossiers de destination pour StyleTTS2
os.makedirs("/content/StyleTTS2/Data", exist_ok=True)
os.makedirs("/content/StyleTTS2/wavs", exist_ok=True)

# 3. Copier les wavs
print("🎵 Copie des fichiers audio vers StyleTTS2...")
source_wavs = os.path.join(source_dir, "wavs")
dest_wavs = "/content/StyleTTS2/wavs"

wav_files = [f for f in os.listdir(source_wavs) if f.endswith('.wav')]
if not wav_files:
    raise FileNotFoundError(f"❌ Aucun fichier .wav trouvé dans {source_wavs}")

# Copie plus sûre
count = 0
for f in wav_files:
    src = os.path.join(source_wavs, f)
    dst = os.path.join(dest_wavs, f)
    # On évite de copier si c'est déjà le même fichier (cas où source = dest)
    if os.path.abspath(src) != os.path.abspath(dst):
        shutil.copy(src, dst)
    count += 1

print(f"✅ {count} fichiers audio prêts.")

# 4. Générer les listes train/val
metadata_path = os.path.join(source_dir, "metadata.json")
with open(metadata_path, 'r', encoding='utf-8') as f:
    metadata = json.load(f)

random.seed(42)
random.shuffle(metadata)

# 95% Train, 5% Val
split = int(len(metadata) * 0.95)
train_data = metadata[:split]
eval_data = metadata[split:]

def write_list(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            # Nettoyage vital pour éviter les erreurs de format (retours à la ligne, pipes)
            text = item['text'].replace('\n', ' ').replace('\r', '').replace('|', '')
            f.write(f"{item['audio_file']}|{text}|0\n")

write_list(train_data, '/content/StyleTTS2/Data/train_list.txt')
write_list(eval_data, '/content/StyleTTS2/Data/val_list.txt')

print(f"✅ Train list : {len(train_data)} lignes")
print(f"✅ Val list : {len(eval_data)} lignes")
print("\n🎉 Préparation terminée ! Vous pouvez lancer l'entraînement.")

🔍 Recherche automatique des données...
✅ Source détectée : /content/darija_dataset
🎵 Copie des fichiers audio vers StyleTTS2...
✅ 1052 fichiers audio prêts.
✅ Train list : 1442 lignes
✅ Val list : 76 lignes

🎉 Préparation terminée ! Vous pouvez lancer l'entraînement.


## 7. Configuration du training

In [24]:
import yaml, os

config = {
    'log_dir': 'Models/Darija',
    'save_freq': 10,
    'device': 'cuda',
    'epochs_1st': 0,
    'epochs_2nd': 80,
    'batch_size': 32,
    'max_len': 400,
    'pretrained_model': 'Models/LibriTTS/epochs_2nd_00020.pth',
    'data_params': {
        'train_data': 'Data/train_list.txt',
        'num_workers': 8,
        'val_data': 'Data/val_list.txt',
        'root_path': '',
        'OOD_data': 'Data/OOD_texts.txt',
        'min_length': 50,
        'sample_rate': 24000,
    },
    'preprocess_params': {
        'sr': 24000,
        'spect_params': {
            'n_fft': 2048,
            'win_length': 1200,
            'hop_length': 300,
            'n_mels': 80
        }
    },
    'model_params': {
        'multispeaker': False,
        'dim_in': 64,
        'hidden_dim': 512,
        'max_conv_dim': 512,
        'n_layer': 3,
        'n_mels': 80,
        'n_token': 178,
        'max_dur': 50,
        'style_dim': 128,
        'dropout': 0.2,
        'decoder': {
            'type': 'istftnet',
            'resblock_kernel_sizes': [3, 7, 11],
            'upsample_rates': [10, 6],
            'upsample_initial_channel': 512,
            'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
            'upsample_kernel_sizes': [20, 12],
            'gen_istft_n_fft': 20,
            'gen_istft_hop_size': 5
        },
        'slm': {
            'model': 'microsoft/wavlm-base-plus',
            'sr': 16000,
            'hidden': 768,
            'nlayers': 13,
            'initial_channel': 64
        },
        'diffusion': {
            'embedding_mask_proba': 0.1,
            'transformer': {
                'num_layers': 3,
                'num_heads': 8,
                'head_features': 64,
                'multiplier': 2
            },
            'dist': {
                'sigma_data': 0.2,
                'estimate_sigma_data': True,
                'mean': -3.0,
                'std': 1.0
            }
        }
    },
    'loss_params': {
        'lambda_mel': 5.0,
        'lambda_gen': 1.0,
        'lambda_slm': 1.0,
        'lambda_mono': 1.0,
        'lambda_s2s': 1.0,
        'lambda_F0': 1.0,
        'lambda_norm': 1.0,
        'lambda_dur': 1.0,
        'lambda_ce': 20.0,
        'lambda_sty': 1.0,
        'lambda_diff': 1.0,
        'diff_epoch': 20,
        'joint_epoch': 40
    },
    'optimizer_params': {'lr': 0.0001},
    'slmadv_params': {
        'min_len': 400,
        'max_len': 500,
        'batch_percentage': 0.5,
        'iter': 10,
        'thresh': 5.0,
        'scale': 0.01,
        'sig': 1.5
    },
    'F0_path': 'Utils/JDC/bst.t7',
    'ASR_config': 'Utils/ASR/config.yml',
    'ASR_path': 'Utils/ASR/epoch_00080.pth',
    'PLBERT_dir': 'Utils/PLBERT/',
}

os.makedirs('/content/StyleTTS2/Configs', exist_ok=True)
with open('/content/StyleTTS2/Configs/config_darija_a100.yml', 'w') as f:
    yaml.dump(config, f)

print("✅ Configuration créée!")

✅ Configuration créée!


In [26]:
import requests
import os
import time

print("📥 Téléchargement depuis les VRAIS liens StyleTTS2...\n")

downloads = [
    ("JDC", "https://github.com/yl4579/StyleTTS2/raw/main/Utils/JDC/bst.t7",
"/content/StyleTTS2/Utils/JDC/bst.t7"),
    ("ASR model", "https://github.com/yl4579/StyleTTS2/raw/main/Utils/ASR/epoch_00080.pth",
"/content/StyleTTS2/Utils/ASR/epoch_00080.pth"),
      ("PLBERT", "https://github.com/yl4579/StyleTTS2/raw/main/Utils/PLBERT/step_1000000.t7",
  "/content/StyleTTS2/Utils/PLBERT/step_1000000.t7"),
    ("PLBERT config", "https://github.com/yl4579/StyleTTS2/raw/main/Utils/PLBERT/config.yml",
"/content/StyleTTS2/Utils/PLBERT/config.yml"),
]

for name, url, path in downloads:
    print(f"📥 {name}...")
    try:
        response = requests.get(url, stream=True, timeout=300, allow_redirects=True)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            size = os.path.getsize(path) / 1024 / 1024
            print(f"✅ {size:.1f} MB\n")
        else:
            print(f"❌ HTTP {response.status_code}\n")
    except Exception as e:
        print(f"❌ {str(e)}\n")
    time.sleep(2)

# Vérification finale
print("\n🔍 Vérification:")
for name, _, path in downloads:
    if os.path.exists(path) and os.path.getsize(path) > 0:
        print(f"✅ {name}: OK")
    else:
        print(f"❌ {name}: MANQUANT")

📥 Téléchargement depuis les VRAIS liens StyleTTS2...

📥 JDC...
✅ 20.1 MB

📥 ASR model...
✅ 90.2 MB

📥 PLBERT...
✅ 24.0 MB

📥 PLBERT config...
✅ 0.0 MB


🔍 Vérification:
✅ JDC: OK
✅ ASR model: OK
✅ PLBERT: OK
✅ PLBERT config: OK


 AJOUT DES FICHIERS MANQUANTS LIBRYTTS ET CONFIG

In [28]:
import os
import shutil
import subprocess

# 1. TÉLÉCHARGER LE MODÈLE MANQUANT (LibriTTS)
print("⏳ Téléchargement du modèle LibriTTS (env. 300Mo)...")
model_dir = "/content/StyleTTS2/Models/LibriTTS"
os.makedirs(model_dir, exist_ok=True)

model_url = "https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Models/LibriTTS/epochs_2nd_00020.pth"
model_path = os.path.join(model_dir, "epochs_2nd_00020.pth")

if not os.path.exists(model_path):
    try:
        subprocess.run(["wget", "-O", model_path, model_url], check=True)
        print("✅ Modèle LibriTTS téléchargé !")
    except Exception as e:
        print(f"❌ Erreur téléchargement : {e}")
else:
    print("ℹ️ Modèle LibriTTS déjà présent.")


# 2. CORRIGER LE NOM DE LA CONFIG
# Le script de vérification cherche "config_darija_ft.yml" mais vous avez "config_darija_a100.yml"
src_config = "/content/StyleTTS2/Configs/config_darija_a100.yml"
dst_config = "/content/StyleTTS2/Configs/config_darija_ft.yml"

if os.path.exists(src_config):
    shutil.copy(src_config, dst_config)
    print(f"✅ Config copiée et renommée : {dst_config}")
else:
    print(f"❌ Config source introuvable : {src_config}")

print("\n🚀 Tout devrait être bon ! Relancez la cellule de vérification.")

⏳ Téléchargement du modèle LibriTTS (env. 300Mo)...
✅ Modèle LibriTTS téléchargé !
✅ Config copiée et renommée : /content/StyleTTS2/Configs/config_darija_ft.yml

🚀 Tout devrait être bon ! Relancez la cellule de vérification.


## 8. Vérification finale avant training

In [29]:
import os
checks = {
    "Config": "/content/StyleTTS2/Configs/config_darija_ft.yml",
    "Audio (wavs)": "/content/StyleTTS2/wavs",
    "Train list": "/content/StyleTTS2/Data/train_list.txt",
    "Val list": "/content/StyleTTS2/Data/val_list.txt",
    "Modèle LibriTTS": "/content/StyleTTS2/Models/LibriTTS/epochs_2nd_00020.pth",
    "JDC": "/content/StyleTTS2/Utils/JDC/bst.t7",
    "ASR": "/content/StyleTTS2/Utils/ASR/epoch_00080.pth",
    "PLBERT": "/content/StyleTTS2/Utils/PLBERT/step_1000000.t7",
}

all_ok = True
for name, path in checks.items():
    exists = os.path.exists(path)
    status = "✅" if exists else "❌"
    print(f"{status} {name}")
    if not exists:
        all_ok = False

# Compter les fichiers
if os.path.exists("/content/StyleTTS2/wavs"):
    wav_count = len([f for f in os.listdir("/content/StyleTTS2/wavs") if f.endswith('.wav')])
    print(f"\n🎵 {wav_count} fichiers audio prêts")

if all_ok:
    print("\n🎉 TOUT EST PRÊT! Vous pouvez lancer le training!")
else:
    print("\n⚠️ Certains fichiers manquent. Vérifiez les étapes précédentes.")

✅ Config
✅ Audio (wavs)
✅ Train list
✅ Val list
✅ Modèle LibriTTS
✅ JDC
✅ ASR
✅ PLBERT

🎵 1052 fichiers audio prêts

🎉 TOUT EST PRÊT! Vous pouvez lancer le training!


In [30]:
# Vérifier que tous les fichiers critiques existent
import os

files_to_check = {
    "Config principal": "/content/StyleTTS2/Configs/config_darija_ft.yml",
    "Train list": "/content/StyleTTS2/Data/train_list.txt",
    "Val list": "/content/StyleTTS2/Data/val_list.txt",
    "Modèle pretrained": "/content/StyleTTS2/Models/LibriTTS/epochs_2nd_00020.pth",
    "JDC": "/content/StyleTTS2/Utils/JDC/bst.t7",
    "ASR model": "/content/StyleTTS2/Utils/ASR/epoch_00080.pth",
    "ASR config": "/content/StyleTTS2/Utils/ASR/config.yml",
    "PLBERT": "/content/StyleTTS2/Utils/PLBERT/step_1000000.t7",
    "PLBERT config": "/content/StyleTTS2/Utils/PLBERT/config.yml"
}

print("🔍 Vérification des fichiers critiques:\n")
all_ok = True

for name, path in files_to_check.items():
    exists = os.path.exists(path)
    size = os.path.getsize(path) if exists else 0
    status = "✅" if exists and size > 0 else "❌"

    if exists and size > 0:
        print(f"{status} {name}: {size/1024/1024:.1f} MB")
    else:
        print(f"{status} {name}: VIDE ou MANQUANT")

    if not exists or size == 0:
        all_ok = False

print(f"\n{'🎉 Tout OK' if all_ok else '⚠️ Fichiers manquants détectés'}")


🔍 Vérification des fichiers critiques:

✅ Config principal: 0.0 MB
✅ Train list: 0.1 MB
✅ Val list: 0.0 MB
✅ Modèle pretrained: 735.7 MB
✅ JDC: 20.1 MB
✅ ASR model: 90.2 MB
✅ ASR config: 0.0 MB
✅ PLBERT: 24.0 MB
✅ PLBERT config: 0.0 MB

🎉 Tout OK


In [31]:
asr_config_optimal = """log_dir: "."
save_freq: 5
log_interval: 10

epochs: 200
batch_size: 16
optimizer: "AdamW"

data_params:
  train_data: ""
  val_data: ""
  root_path: ""
  OOD_data: ""
  min_length: 50
  max_length: 500

loss_params:
  lambda_mel: 5.0

optimizer_params:
  lr: 0.0001
  betas: [0.9, 0.99]
  weight_decay: 0.0

model_params:
  dim_in: 80
  hidden_dim: 512
  n_token: 178
  token_embedding_dim: 512

preprocess_params:
  sr: 24000
  spect_params:
    n_fft: 2048
    win_length: 1200
    hop_length: 300
    n_mels: 80
"""

# Sauvegarder
with open('/content/StyleTTS2/Utils/ASR/config.yml', 'w') as f:
    f.write(asr_config_optimal)

print("✅ Config ASR optimal créé\n")


# -----------------------------------------------------
# PLBERT CONFIG
# -----------------------------------------------------

plbert_config = """log_dir: "."
save_freq: 2
log_interval: 10
device: "cuda"
epochs: 50
batch_size: 256

model_params:
  model_type: "bert-base-uncased"
  hidden_size: 768
  num_hidden_layers: 12
  num_attention_heads: 12

data_params:
  train_data: ""
  val_data: ""

optimizer_params:
  lr: 0.0001
"""

with open('/content/StyleTTS2/Utils/PLBERT/config.yml', 'w') as f:
    f.write(plbert_config)

print("✅ Config PLBERT créé\n")


# -----------------------------------------------------
# VÉRIFICATION COMPLÈTE
# -----------------------------------------------------

import os

print("🔍 VÉRIFICATION FINALE COMPLÈTE:\n")

critical_files = {
    "Config Training": "/content/StyleTTS2/Configs/config_darija_ft.yml",
    "Train List": "/content/StyleTTS2/Data/train_list.txt",
    "Val List": "/content/StyleTTS2/Data/val_list.txt",
    "Pretrained Model": "/content/StyleTTS2/Models/LibriTTS/epochs_2nd_00020.pth",
    "LibriTTS Config": "/content/StyleTTS2/Models/LibriTTS/config.yml",
    "JDC Model": "/content/StyleTTS2/Utils/JDC/bst.t7",
    "ASR Model": "/content/StyleTTS2/Utils/ASR/epoch_00080.pth",
    "ASR Config": "/content/StyleTTS2/Utils/ASR/config.yml",
    "PLBERT Model": "/content/StyleTTS2/Utils/PLBERT/step_1000000.t7",
    "PLBERT Config": "/content/StyleTTS2/Utils/PLBERT/config.yml"
}

all_good = True

for name, path in critical_files.items():
    if os.path.exists(path):
        size = os.path.getsize(path)
        if size > 0:
            size_mb = size / 1024 / 1024
            print(f"✅ {name}: {size_mb:.2f} MB")
        else:
            print(f"❌ {name}: VIDE (0 bytes)")
            all_good = False
    else:
        print(f"❌ {name}: MANQUANT")
        all_good = False


# Compter les fichiers audio
wavs_dir = "/content/StyleTTS2/wavs"
if os.path.exists(wavs_dir):
    wav_files = [f for f in os.listdir(wavs_dir) if f.endswith('.wav')]
    print(f"\n🎵 Fichiers audio: {len(wav_files)} fichiers WAV")
else:
    print("\n❌ Dossier wavs manquant!")
    all_good = False


print(f"\n{'🎉 TOUT EST PRÊT POUR LE TRAINING!' if all_good else '⚠️ FICHIERS MANQUANTS - VOIR CI-DESSUS'}")


✅ Config ASR optimal créé

✅ Config PLBERT créé

🔍 VÉRIFICATION FINALE COMPLÈTE:

✅ Config Training: 0.00 MB
✅ Train List: 0.10 MB
✅ Val List: 0.01 MB
✅ Pretrained Model: 735.66 MB
❌ LibriTTS Config: MANQUANT
✅ JDC Model: 20.06 MB
✅ ASR Model: 90.17 MB
✅ ASR Config: 0.00 MB
✅ PLBERT Model: 24.02 MB
✅ PLBERT Config: 0.00 MB

🎵 Fichiers audio: 1052 fichiers WAV

⚠️ FICHIERS MANQUANTS - VOIR CI-DESSUS


## 9. 🚀 Lancement du Fine-Tuning

⚠️ **Durée estimée: 8-12h sur GPU T4**

Checkpoints sauvegardés tous les 10 epochs dans `/content/StyleTTS2/Models/Darija/`

In [32]:
# Cell: Create PLBERT config that matches checkpoint
import yaml
import os

plbert_config = {
    'model_params': {
        'vocab_size': 178,           # Must match checkpoint (not 30000)
        'hidden_size': 768,
        'num_hidden_layers': 12,
        'num_attention_heads': 12,
        'intermediate_size': 2048,   # Must match checkpoint (not 16384)
        'hidden_act': 'gelu',
        'hidden_dropout_prob': 0.0,
        'attention_probs_dropout_prob': 0.0,
        'max_position_embeddings': 512,
        'type_vocab_size': 2,
        'initializer_range': 0.02,
        'layer_norm_eps': 1e-12
    }
}

# Save PLBERT config
plbert_config_path = '/content/StyleTTS2/Utils/PLBERT/config.yml'
os.makedirs(os.path.dirname(plbert_config_path), exist_ok=True)

with open(plbert_config_path, 'w') as f:
    yaml.dump(plbert_config, f, default_flow_style=False)

print(f"✅ PLBERT config saved to {plbert_config_path}")
print("\nConfig:")
with open(plbert_config_path, 'r') as f:
     print(f.read())

✅ PLBERT config saved to /content/StyleTTS2/Utils/PLBERT/config.yml

Config:
model_params:
  attention_probs_dropout_prob: 0.0
  hidden_act: gelu
  hidden_dropout_prob: 0.0
  hidden_size: 768
  initializer_range: 0.02
  intermediate_size: 2048
  layer_norm_eps: 1.0e-12
  max_position_embeddings: 512
  num_attention_heads: 12
  num_hidden_layers: 12
  type_vocab_size: 2
  vocab_size: 178



In [38]:
# 🚀 LANCEMENT ROBUSTE DU TRAINING
%cd /content/StyleTTS2

import torch
import gc
import os
import yaml

# 1. Nettoyage mémoire
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

# 2. Forcer une config ASR propre (Correction Erreur "dim_in")
asr_config_path = "Utils/ASR/config.yml"
asr_clean_config = {
    "model_params": {
        "input_dim": 80,
        "hidden_dim": 256,
        "n_token": 178,     # Votre valeur spécifique
        "n_layers": 6,
        "token_embedding_dim": 256 # Valeur standard compatible
    }
}

print(f"🛠️ Réparation de {asr_config_path}...")
os.makedirs(os.path.dirname(asr_config_path), exist_ok=True)
with open(asr_config_path, 'w') as f:
    yaml.dump(asr_clean_config, f, default_flow_style=False)
print("✅ Config ASR régénérée.")

# 3. Lancement
print("🏁 Démarrage de l'entraînement...")
!python train_finetune.py --config_path Configs/config_darija_ft.yml

/content/StyleTTS2
🛠️ Réparation de Utils/ASR/config.yml...
✅ Config ASR régénérée.
🏁 Démarrage de l'entraînement...
2025-12-10 14:36:30.045553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765377390.080038   11392 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765377390.090436   11392 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765377390.114823   11392 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765377390.114858   11392 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking th

## 10. Sauvegarde sur Google Drive (IMPORTANT!)

In [39]:
# 🚀 LANCEMENT ROBUSTE DU TRAINING (CORRECTION V2)
%cd /content/StyleTTS2

import torch
import gc
import os
import yaml

# 1. Nettoyage mémoire
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

# 2. Forcer une config ASR propre (CORRECTED)
asr_config_path = "Utils/ASR/config.yml"
asr_clean_config = {
    "model_params": {
        "input_dim": 80,
        "hidden_dim": 256,
        "n_token": 178,
        "n_layers": 6,
        "token_embedding_dim": 512  # <--- CORRECTION ICI (512 au lieu de 256)
    }
}

print(f"🛠️ Réparation V2 de {asr_config_path}...")
os.makedirs(os.path.dirname(asr_config_path), exist_ok=True)
with open(asr_config_path, 'w') as f:
    yaml.dump(asr_clean_config, f, default_flow_style=False)
print("✅ Config ASR régénérée (Embedding 512).")

# 3. Lancement
print("🏁 Démarrage de l'entraînement...")
!python train_finetune.py --config_path Configs/config_darija_ft.yml

/content/StyleTTS2
🛠️ Réparation V2 de Utils/ASR/config.yml...
✅ Config ASR régénérée (Embedding 512).
🏁 Démarrage de l'entraînement...
2025-12-10 14:42:18.937410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765377738.969772   12850 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765377738.979734   12850 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765377739.003803   12850 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765377739.003838   12850 computation_placer.cc:177] computation placer already registered. Please check linkage a

In [36]:
from google.colab import drive
import shutil
import os

# Monter Google Drive
drive.mount('/content/drive')

# Créer le dossier de sauvegarde
backup_dir = '/content/drive/MyDrive/darija_checkpoints'
os.makedirs(backup_dir, exist_ok=True)

# Copier les checkpoints
print("💾 Sauvegarde des checkpoints sur Google Drive...")
source_dir = '/content/StyleTTS2/Models/Darija'
if os.path.exists(source_dir):
    for item in os.listdir(source_dir):
        source_path = os.path.join(source_dir, item)
        dest_path = os.path.join(backup_dir, item)
        if os.path.isfile(source_path):
            shutil.copy2(source_path, dest_path)
            print(f"✅ {item}")
    print("\n🎉 Checkpoints sauvegardés sur Drive!")
else:
    print("⚠️ Aucun checkpoint trouvé")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
💾 Sauvegarde des checkpoints sur Google Drive...
⚠️ Aucun checkpoint trouvé


## 11. Upload du modèle entraîné sur Hugging Face (optionnel)

In [ ]:
from huggingface_hub import HfApi, login

# Configuration
HF_MODEL_REPO = "VOTRE-USERNAME/darija-styletts2"  # Changez ici
HF_TOKEN_UPLOAD = ""  # Votre token HF avec droits write

if HF_TOKEN_UPLOAD:
    print("🔐 Connexion à Hugging Face...")
    login(token=HF_TOKEN_UPLOAD)

    api = HfApi()

    # Créer le repo
    api.create_repo(repo_id=HF_MODEL_REPO, repo_type="model", exist_ok=True)

    # Upload les checkpoints
    print(f"📤 Upload vers {HF_MODEL_REPO}...")
    api.upload_folder(
        folder_path="/content/StyleTTS2/Models/Darija",
        repo_id=HF_MODEL_REPO,
        repo_type="model"
    )

    print(f"✅ Modèle disponible sur: https://huggingface.co/{HF_MODEL_REPO}")
else:
    print("ℹ️ Token HF non fourni, upload ignoré")